In [1]:
from codes.trajectory import Trajectory
import pandas as pd
import numpy as np
import h5py
import folium
import pickle
import os 
import json
from itertools import groupby

%load_ext autoreload
%autoreload 2

In [2]:
pd.set_option('max_row',400)

In [3]:
import datetime

In [4]:
data_path = '/data3/zhuzheng/trajectory/porto/porto2.pkl'
df = pd.read_pickle(data_path)

In [159]:
test_idx = 12
test_traj = df.loc[test_idx, 'POLYLINE']
s_time = df.loc[test_idx, 'TIMESTAMP']
test_timestamp = [s_time + i * 15 for i in range(len(test_traj))]

In [166]:
traj = Trajectory(np.array(test_traj), np.array(test_timestamp), '1', turning_threshold=0.3, use_graph_feature=True)

In [167]:
traj.get_basic_feature()

[False,
 12.155256556193308,
 57.509471381413285,
 1.5394814290342433,
 12.786982249711574,
 22.484239526915495,
 0.0008111002584508531,
 0.1914138086235,
 -0.5674092421405924,
 0.004407233333528638,
 0.39871553010481464,
 13.611023189080715,
 58.28554239547212,
 2.8049028925648543,
 13.511100915608122,
 22.089939775820596,
 0.013007742302792336,
 0.24158552710636283,
 -0.546095278090864,
 -0.02256456117150011,
 0.4021212529072332]

In [168]:
traj.get_graph_feature()

[0.8888888888888888,
 0.1111111111111111,
 0.7777777777777778,
 0.0,
 0.8888888888888888,
 0.2222222222222222,
 0.5,
 0.0,
 0.25,
 0.25,
 1.0,
 0.25,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.25,
 1.0,
 0.0,
 0.25,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.25,
 0.0,
 0.5,
 0.25,
 0.0,
 0.0,
 0.75,
 0.5,
 0.25,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0]

In [169]:
traj.vis(with_marker=True)

## Sample a small set for feature test

In [7]:
df_sample = df.sample(100000)

In [8]:
df_sample['LABEL'] = pd.factorize(df_sample['TAXI_ID'])[0].astype(np.uint16)

In [68]:
df_sample.set_index(np.arange(df_sample.shape[0]), inplace=True)

In [69]:
df_sample.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,INDEX,DISTANCE,SD,LABEL
0,1376122623620000626,A,54236.0,NaN,20000626,1376122623,A,False,"[[-8.652357, 41.169231], [-8.65233, 41.169222]...",184257,3363.689740,ez3f5n:ez3f55,0
1,1403012163620000327,C,NaN,NaN,20000327,1403012163,A,False,"[[-8.58564, 41.148576], [-8.585748, 41.148675]...",1636792,11515.183204,ez3fhd:ez3f4y,1
2,1374839276620000607,A,8470.0,NaN,20000607,1374839276,A,False,"[[-8.610993, 41.177835], [-8.610993, 41.177745...",120644,10439.397221,ez3fk0:ez3fej,2
3,1402310469620000330,B,NaN,27.0,20000330,1402310469,A,False,"[[-8.608725, 41.147667], [-8.608725, 41.147676...",1598458,1119.479915,ez3fh4:ez3f5c,3
4,1377321595620000904,C,NaN,NaN,20000904,1377321595,A,False,"[[-8.604009, 41.154678], [-8.60463, 41.154714]...",239717,28373.676741,ez3fh5:ez3f6c,4


In [70]:
df_sample = df_sample[['LABEL', 'TIMESTAMP', 'DISTANCE', 'SD', 'TAXI_ID', 'POLYLINE']]

In [71]:
df_sample.to_pickle('/data3/zhuzheng/trajectory/porto/porto2_sample10k.pkl')

# Test feature engineering

In [5]:
from codes.trajectory_dataset import TrajectoryDataset

In [104]:
df_sample['CoordN'] = df_sample.apply(lambda df: len(df['POLYLINE']), axis=1)

In [105]:
df_sample_ = df_sample[df_sample['CoordN'] > 4]

In [6]:
traj_datas = TrajectoryDataset(df, turning_threshold=0.3)

In [200]:
traj_datas.df_feature = None
df_feature = traj_datas.generate_trajectories_feature()

In [201]:
df_feature.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,LABEL
0,False,10.678380,50.216279,0.201459,10.406066,19.944462,0.282024,0.077227,0.033526,0.143939,0.682731,-0.308883,0.057235,-0.551610,-0.251420,-0.013804,12.062500,35.116023,5.665130,10.780580,19.522734,0.359014,0.097465,0.080355,0.143939,0.748382,-0.324787,0.055241,-0.601417,-0.251420,-0.098382,0.0
1,False,13.235843,99.003725,0.109601,11.959481,27.404068,0.189042,0.028317,0.019817,0.125194,0.453731,-0.202346,0.031449,-0.358582,-0.190463,-0.008311,11.326285,105.379324,0.120779,8.129347,27.480851,0.187949,0.024160,0.032282,0.141101,0.358724,-0.240026,0.041218,-0.371653,-0.283782,-0.013751,1.0
2,False,12.209821,83.495995,0.342086,12.474355,25.291158,0.220365,0.030704,0.053720,0.204874,0.454588,-0.184679,0.062839,-0.553156,-0.053975,-0.000617,11.893742,78.785475,0.346541,10.737677,25.114583,0.216947,0.025907,0.086109,0.181868,0.380741,-0.190780,0.047346,-0.507754,-0.056804,-0.015657,2.0
3,False,2.132343,5.137509,0.180196,1.402936,5.838940,0.125290,0.012160,0.014866,0.073475,0.288410,-0.103628,0.013805,-0.295896,-0.063046,-0.002187,1.802197,4.165121,0.201329,1.105799,5.783353,0.090191,0.008180,0.009713,0.066579,0.210426,-0.112891,0.016571,-0.300955,-0.072535,-0.005764,3.0
4,False,18.616679,138.600833,2.884819,17.977809,33.972847,0.318385,0.074694,0.053762,0.227095,0.585746,-0.360221,0.112093,-0.676525,-0.299761,-0.045813,15.484869,122.030384,1.875907,13.873753,31.355114,0.336163,0.063884,0.138395,0.247630,0.636687,-0.435434,0.123056,-0.718790,-0.362026,-0.045813,4.0


In [202]:
df_feature_ = df_feature[~df_feature[0].values]

In [203]:
df_feat_data = df_feature_.values[:,1:-1]

In [204]:
df_feat_data.shape

(96841, 30)

In [205]:
labels = df_feature_['LABEL'].values.astype(int)

In [129]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [206]:
min_max_scaler = preprocessing.MinMaxScaler()
df_feat_data_scaled = min_max_scaler.fit_transform(df_feat_data)
# df_feat_data_norm = preprocessing.normalize(df_feat_data, 'l2')

/home/zhuzheng/env/py36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [207]:
X_train, X_test, y_train, y_test = train_test_split(df_feat_data_scaled, labels)

In [210]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [148]:
# original use all features
# pred = tree_clf.predict(X_test)
# accuracy_score(y_test, pred)

0.0033868902564949815

In [211]:
pred = tree_clf.predict(X_test)
accuracy_score(y_test, pred)

0.003717318574201809

In [212]:
tree_clf.feature_importances_

array([0.03105646, 0.03080581, 0.03412263, 0.03353961, 0.03198657,
       0.03243477, 0.03426742, 0.03355194, 0.03491424, 0.03365054,
       0.03147179, 0.03425224, 0.03475724, 0.03532214, 0.03558131,
       0.03229286, 0.03211821, 0.03304335, 0.03467769, 0.03317595,
       0.03182022, 0.03282636, 0.03587415, 0.0333942 , 0.0337717 ,
       0.03125865, 0.03242587, 0.03458914, 0.0326254 , 0.03439155])

In [213]:
print('driver number in sampled data',len(set(labels)))

driver number in sampled data 440


In [215]:
traj_datas.driver_statistics()

,Speed_Mean,Speed_Var,Ac_Mean,Ac_Var,Dc_Mean,Dc_Var,Steer_Speed_Mean,Steer_Speed_Var,Steer_Ac_Mean,Steer_Ac_Var,Steer_Dc_Mean,Steer_Dc_var,DISTANCE
LABEL,,,,,,,,,,,,,
0.0,7.169460,36.418592,0.220622,0.040450,-0.214072,0.049491,6.259360,30.844402,0.195603,0.033930,-0.233792,0.045771,5967.106819
1.0,7.139026,35.265062,0.217917,0.039642,-0.208170,0.046449,6.175085,28.730397,0.194620,0.035022,-0.223450,0.044643,8043.138260
2.0,7.071596,34.188460,0.225668,0.043383,-0.213540,0.051587,6.051474,27.931632,0.194526,0.032965,-0.232843,0.044813,6055.808334
3.0,7.382516,36.180831,0.215238,0.045240,-0.203342,0.046980,6.237271,28.786792,0.208973,0.032195,-0.225468,0.042583,4647.294155
4.0,7.260001,33.562638,0.222357,0.039337,-0.215383,0.046516,6.336616,26.702456,0.197344,0.032565,-0.231574,0.043169,12825.641172
5.0,7.024837,33.109313,0.223206,0.042686,-0.212536,0.047082,6.058938,27.408386,0.200281,0.036903,-0.234275,0.046331,5961.221528
6.0,6.914456,33.365674,0.224514,0.037736,-0.209097,0.043998,6.039028,27.912381,0.206798,0.032599,-0.233495,0.041406,4624.263903
7.0,7.659865,38.582689,0.225166,0.041209,-0.214521,0.049346,6.566292,31.246017,0.197972,0.033477,-0.232317,0.043494,4677.621109
8.0,7.091090,34.697565,0.219550,0.036972,-0.207851,0.044852,6.193118,29.525217,0.193795,0.031476,-0.226847,0.042501,5911.466464


## Driver analysis

In [26]:
driver = traj_datas.get_driver(16)

In [27]:
driver.driver_date_records()

DATE
2013-07-02    1
2013-07-05    1
2013-07-06    1
2013-07-10    2
2013-07-11    1
2013-07-12    1
2013-07-14    1
2013-07-15    1
2013-07-16    1
2013-07-17    1
2013-07-18    1
2013-07-19    2
2013-07-21    2
2013-07-25    1
2013-07-26    1
2013-07-27    1
2013-07-28    1
2013-07-31    2
2013-08-01    2
2013-08-03    1
2013-08-05    1
2013-08-06    1
2013-08-07    1
2013-08-08    1
2013-08-10    1
2013-08-11    1
2013-08-16    1
2013-08-17    1
2013-08-20    3
2013-08-21    1
2013-08-24    1
2013-08-27    1
2013-09-04    1
2013-09-08    2
2013-09-12    2
2013-09-13    2
2013-09-16    1
2013-09-21    1
2013-09-23    1
2013-09-25    2
2013-10-03    1
2013-10-09    1
2013-10-11    1
2013-10-13    2
2013-10-14    2
2013-10-16    2
2013-10-18    1
2013-10-19    2
2013-10-20    2
2013-10-22    1
2013-10-23    2
2013-10-24    3
2013-10-25    1
2013-10-26    2
2013-10-27    1
2013-10-29    1
2013-11-02    2
2013-11-04    1
2013-11-06    2
2013-11-11    1
2013-11-13    2
2013-11-24    2
201

In [12]:
driver.vis_trajs_hour(40, [9,15], with_marker=False)

In [29]:
driver.vis_trajs_date(30, [2014, 5, 25], with_marker=False)

## OD Analysis

In [15]:
traj_datas.od_num_info()

SD
ez3fh4:ez3fh4    779
ez3fh4:ez3f5f    633
ez3fhd:ez3fh4    556
ez3f5c:ez3fh4    549
ez3f5s:ez3fh4    548
ez3fh4:ez3f5s    530
ez3f5f:ez3fh4    472
ez3fh4:ez3fhh    454
ez3fh4:ez3fhj    401
ez3fh4:ez3fhd    380
ez3fh4:ez3f5e    378
ez3fh4:ez3fh5    366
ez3fh4:ez3fde    365
ez3fh4:ez3fdd    333
ez3fhj:ez3fh4    329
ez3fh5:ez3fh5    318
ez3f5s:ez3f5f    310
ez3fh4:ez3f5k    307
ez3fh4:ez3f5g    307
ez3f5f:ez3f5f    275
ez3fh5:ez3fh4    260
ez3f5f:ez3f5s    259
ez3f5f:ez3fhh    258
ez3fh4:ez3f55    253
ez3fh5:ez3f5f    245
ez3fh4:ez3f5c    244
ez3fh4:ez3fhm    241
ez3f5f:ez3fh5    240
ez3f5c:ez3f5f    238
ez3fh4:ez3fhk    235
ez3fh5:ez3fhj    234
ez3fh4:ez3f5m    223
ez3fh4:ez3f5t    222
ez3f5c:ez3fh5    221
ez3f55:ez3f55    220
ez3f5f:ez3f5e    219
ez3f5s:ez3fh5    218
ez3fhd:ez3fh5    218
ez3f5k:ez3fh4    211
ez3fhd:ez3f5s    210
ez3f55:ez3f5f    209
ez3fh4:ez3fh7    208
ez3fh5:ez3f5s    206
ez3f55:ez3fh4    204
ez3fh1:ez3fh4    202
ez3f5f:ez3f5k    185
ez3f5s:ez3f5s    184
ez3f5f:ez3

In [10]:
od1 = traj_datas.get_od('ez3fh4:ez3cup')

In [23]:
od1.vis_trajs_hour(5, [12,13], with_marker=False)

In [26]:
od1.show_passed_driver()

LABEL
0       2
1       2
2       3
5       1
6       1
7       4
8       3
9       7
11      5
12      3
13      2
15     17
16      1
17      4
18      5
19      7
20      9
21      6
22      1
23      8
24      2
25      3
26     10
27     25
28     20
32      1
34      4
36      9
37      2
38      8
39      5
40      6
41      3
42     14
43      5
46      5
47      8
48     10
49      1
51      4
52     10
53      6
54      1
55      8
58      6
59      5
60      6
61      3
62      7
63      4
66      5
68      1
70      5
71      4
72      2
73      4
74      2
76      4
77      6
78      3
80      2
81     17
82      3
83      5
85      1
86      2
88      1
89      9
90      6
91      4
92      1
93      5
94      2
95      1
96      4
97      3
98      1
99      1
100     1
102     2
104    10
105     3
107     5
108     3
109    19
110     2
111     2
113     5
114     9
117     5
118     6
119     3
120     1
121     4
123     7
125     1
126     7
127     5
128     2
129 

In [40]:
od1.vis_two_driver(109, 191, 2, 2, with_marker=True)